In [ ]:
# import os
# import shutil

# # Create the folder in your Windows user directory
# kaggle_path = os.path.expanduser('~/.kaggle')
# if not os.path.exists(kaggle_path):
#     os.makedirs(kaggle_path)

# # Copy the file (make sure kaggle.json is in your current folder)
# shutil.copy('kaggle.json', os.path.join(kaggle_path, 'kaggle.json'))

# print("Kaggle folder set up successfully!")

Kaggle folder set up successfully!


In [ ]:
# # The '!' tells VS Code to run this in your terminal/command prompt
# !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
  4%|▍         | 1.00M/25.7M [00:01<00:39, 663kB/s]
  8%|▊         | 2.00M/25.7M [00:01<00:18, 1.33MB/s]
 12%|█▏        | 3.00M/25.7M [00:01<00:11, 2.12MB/s]
 16%|█▌        | 4.00M/25.7M [00:02<00:07, 3.00MB/s]
 19%|█▉        | 5.00M/25.7M [00:02<00:05, 3.99MB/s]
 23%|██▎       | 6.00M/25.7M [00:02<00:04, 4.97MB/s]
 27%|██▋       | 7.00M/25.7M [00:02<00:03, 6.00MB/s]
 31%|███       | 8.00M/25.7M [00:02<00:02, 6.86MB/s]
 35%|███▌      | 9.00M/25.7M [00:02<00:02, 7.24MB/s]
 43%|████▎     | 11.0M/25.7M [00:02<00:01, 7.75MB/s]
 47%|████▋     | 12.0M/25.7M [00:02<00:01, 8.26MB/s]
 51%|█████     | 13.0M/25.7M [00:03<00:01, 8.72MB/s]
 54%|█████▍    | 14.0M/25.7M [00:03<00:01, 9.04MB/s]
 58%|█████▊    | 15.0M/25.7M [00:03<00:01, 9.29MB/s]
 62%|██████▏   | 16.0M/25.7M [00:03<00:01, 9.32MB/s]
 66%|██████▌   | 17.0M/25.7M [00:03<00:01, 8.89MB/s]
 70%|███████   | 18.0M/25.7M [00:03<00:00, 9.21MB/s]
 74%|███████▍  | 19.0M/25.7M [00:03<00:00, 9.35MB/s]
 7

In [ ]:
import pandas as pd
import numpy as np


In [9]:
df = pd.read_csv('D:\\Learning\\Test\\RNN\\New\\IMDB Dataset.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [15]:
df.isnull().sum()


review       0
sentiment    0
dtype: int64

In [16]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [19]:
y = df['sentiment']
X = df['review']

In [20]:
y = np.where(df['sentiment']=='positive',1,0)

#### Lowercase

In [32]:
X = X.str.lower()

#### Remove Punctuation

In [34]:
import string
X = X.str.replace('{string.punctuation}',' ',regex=True)  

#### Remove Number

In [38]:
X = X.str.replace(r'\d+',' ',regex=True)

#### Remove URL Links

In [41]:
X = X.str.replace(r'http\s+|www\s+|https\s+',' ',regex=True)

#### Remove Emoji and extra character

In [43]:
X = X.str.replace(r'^a-zA-Z0-9\s',' ',regex=True)

#### Remove BR tag

In [46]:
X = X.str.replace('<br />',' ',regex=True)


In [49]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [50]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X)

In [63]:
volcab_size = len(tokenizer.word_index)+1
X_seq = tokenizer.texts_to_sequences(X)

In [ ]:
tokenizer.word_index

In [64]:
X[0]

"one of the other reviewers has mentioned that after watching just   oz episode you'll be hooked. they are right, as this is exactly what happened with me.  the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.  it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.  i would say the main appeal of the show is due to the fact that it goes where other sh

In [94]:
seq_lengths = [len(x) for x in X_seq] 
count = 0 
seq_lengths_sorted = sorted(seq_lengths, reverse=True)
for i in seq_lengths:
    if i > 300:
        count+=1

count


11010

In [95]:
max_len = 300

In [96]:
X_padded = pad_sequences(X_seq,maxlen=max_len,padding='post',truncating='post')

## Model

In [111]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from tensorflow.keras.layers import Dropout
model = Sequential()
model.add(Embedding(input_dim=volcab_size, output_dim=100, input_shape=(max_len,)))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))


In [112]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 300, 100)       │    12,195,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 300, 150)       │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 300, 150)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 150)            │       180,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           151 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,527,251 (47.79 MB)

 Trainable params: 12,527,251 (47.79 MB)

 Non-trainable params: 0 (0.00 B)

In [113]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_padded, y, epochs=10, batch_size=64, validation_split=0.2)

In [ ]:
import pickle

model.keras.save("IMDB.keras")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)